# TensorBoard

TensorBoard is a great interactive visualization tool that you can use to view the learning curves during training, compare learning curves between multiple runs, visualize the computation graph, analyze training statistics, view images generated by your model, visualize complex multidimensional data projected down to 3D and automatically clustered for you, and more! This tool is installed automatically when
you install TensorFlow, so you already have it!

To use it, you must modify your program so that it outputs the data you want to visualize to special binary log files called event files. Each binary data record is called a summary. The TensorBoard server will monitor the log directory, and it will automatically pick up the changes and update the visualizations: this allows you to visualize
live data (with a short delay), such as the learning curves during training. In general, you want to point the TensorBoard server to a root log directory, and configure your program so that it writes to a different subdirectory every time it runs. This way, the same TensorBoard server instance will allow you to visualize and compare data from multiple runs of your program, without getting everything mixed up.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

To start with let's define the root log directory

In [6]:
import os as os
root_logdir = os.path.join(os.curdir, "my_logs")

Next we will define a small function that will generate a subdirectory path based on the current date and time, so that it is different at every run. You may want to include extra information in the log directory name, such as hyperparameter values that you are testing, to
make it easier to know what you are looking at in TensorBoard

In [8]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2022_09_27-21_36_09'

In [9]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

2022-09-27 21:36:35.190188: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-27 21:36:35.192071: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


TensorBoard is a callback, which can be passed as any other callback while training

In [11]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 3s 8ms/step - loss: 1.8866 - val_loss: 0.7126
Epoch 2/30
363/363 [==============================] - 3s 8ms/step - loss: 0.6577 - val_loss: 0.6880
Epoch 3/30
363/363 [==============================] - 3s 8ms/step - loss: 0.5934 - val_loss: 0.5803
Epoch 4/30
363/363 [==============================] - 3s 9ms/step - loss: 0.5557 - val_loss: 0.5166
Epoch 5/30
363/363 [==============================] - 3s 8ms/step - loss: 0.5272 - val_loss: 0.4895
Epoch 6/30
363/363 [==============================] - 3s 8ms/step - loss: 0.5033 - val_loss: 0.4951
Epoch 7/30
363/363 [==============================] - 3s 8ms/step - loss: 0.4854 - val_loss: 0.4861
Epoch 8/30
363/363 [==============================] - 4s 10ms/step - loss: 0.4709 - val_loss: 0.4554
Epoch 9/30
363/363 [==============================] - 3s 7ms/step - loss: 0.4578 - val_loss: 0.4413
Epoch 10/30
363/363 [==============================] - 3s 9ms/step - loss: 0.4474 - val_loss: 0.437

In [12]:
X_new = X_test[:3]
y_pred = model.predict(X_new)
print(y_pred, y_test[:3])

[[0.58085257]
 [1.9019513 ]
 [3.607911  ]] [0.477   0.458   5.00001]


In [13]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

Alternatively, go to this notebook's directory, then type:

$ tensorboard --logdir=./my_logs --port=6006

You can then open your web browser to localhost:6006 and use TensorBoard. Once you are done, press Ctrl-C in the terminal window, this will shutdown the TensorBoard server.

In [14]:
run_logdir2 = get_run_logdir()
run_logdir2

'./my_logs/run_2022_09_27-21_51_17'

In [15]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=0.05))

In [17]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir2)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 3s 8ms/step - loss: 0.4493 - val_loss: 17.9525
Epoch 2/30
363/363 [==============================] - 3s 8ms/step - loss: 12.4573 - val_loss: 2.4107
Epoch 3/30
363/363 [==============================] - 2s 6ms/step - loss: 0.9406 - val_loss: 0.4016
Epoch 4/30
363/363 [==============================] - 2s 7ms/step - loss: 0.4264 - val_loss: 0.4303
Epoch 5/30
363/363 [==============================] - 2s 5ms/step - loss: 0.4283 - val_loss: 0.3740
Epoch 6/30
363/363 [==============================] - 2s 4ms/step - loss: 0.4150 - val_loss: 0.3954
Epoch 7/30
363/363 [==============================] - 2s 5ms/step - loss: 0.4006 - val_loss: 0.3349
Epoch 8/30
363/363 [==============================] - 3s 8ms/step - loss: 0.3799 - val_loss: 0.3576
Epoch 9/30
363/363 [==============================] - 3s 8ms/step - loss: 0.3711 - val_loss: 0.3323
Epoch 10/30
363/363 [==============================] - 2s 6ms/step - loss: 0.3779 - val_loss: 0.37

Note you can see both the iteration by refreshing the TensorBoard